In [10]:
%load_ext autoreload
%autoreload 2

import os, sys
import copy
import socket
from tqdm import tqdm
import torch
import pickle
from torch import optim
from torch.utils.tensorboard import SummaryWriter

#!pip install networkx matplotlib
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
from libs.topology_manager import *
from libs import fl, nn, data, log

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Save Logs To File (info | debug | warning | error | critical) [optional]
log.init("info")
#log.init("info", "federated.log")
#log.init("debug", "flkafka.log")

In [12]:
adj_mat = np.array([[0,1,1,1,1,1],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0], 
             [1,0,0,0,0,0]])

node_types = {'aggregator': [0,1], 'trainer': [2,3], 'broadcaster': [4,5]}

di_graph = nx.DiGraph(adj_mat)
#nx.draw(di_graph)

In [13]:
class FedArgs():
    def __init__(self):
        self.num_clients = len(adj_mat)
        self.epochs = 10
        self.local_rounds = 1
        self.client_batch_size = 32
        self.test_batch_size = 128
        self.learning_rate = 1e-4
        self.weight_decay = 1e-5
        self.cuda = False
        self.seed = 1
        self.topic = 'pyflx-c'
        self.server_topic = "pyflx-cs"
        self.tb = SummaryWriter('../out/runs/flkafka', comment="Mnist Distributed Federated training")

fedargs = FedArgs()

In [14]:
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [15]:
host = socket.gethostname()
server = host + "(server)"
clients = [host + "(" + str(client + 1) + ")" for client in range(fedargs.num_clients)]
ctp = CentralizedTopology(adj_mat, server, clients, node_types)

In [16]:
# Load MNIST Data to clients
train_data, test_data = data.load_dataset("mnist")
clients_data = data.split_data(train_data, clients)
client_train_loaders, client_test_loaders = data.load_client_data(clients_data, fedargs.client_batch_size, 0.2, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=fedargs.test_batch_size, shuffle=True, **kwargs)

clients_info = {
        client: {"train_loader": client_train_loaders[client],
                 "test_loader": client_test_loaders[client]}
        for client in clients
    }

In [17]:
import asyncio
import time

def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def process(client, epoch, ctp, model, train_loader, test_loader, fedargs, device):
    log.info("Processing Client {}".format(client))

    # Consume and Average, epoch passed is actually prev epoch, for which we want to consume updates
    rcvd_models = ctp.consume_model(client, fedargs.server_topic, model, epoch)
    log.info("Client {} received {} model updates from {}".format(client, len(rcvd_models), list(rcvd_models.keys())))
    if len(rcvd_models) != 0:
        model = fl.federated_avg(rcvd_models)

    # Train  
    model, loss = fl.client_update(model,
                                train_loader,
                                fedargs.learning_rate,
                                fedargs.weight_decay,
                                fedargs.local_rounds,
                                device)
    
    ctp.produce_model(client, fedargs.topic, model, epoch)

    # Plot and Log
    for local_epoch, loss in enumerate(list(loss.values())):
        fedargs.tb.add_scalars("Training Loss/" + client, {str(epoch): loss}, str(local_epoch + 1))

    log.jsondebug(loss, "Epoch {} of {} : Federated Training loss, Client {}".format(epoch, fedargs.epochs, client))
    log.modeldebug(model, "Epoch {} of {} : Client {} Update".format(epoch, fedargs.epochs, client))

    # Test
    test_output = fl.eval(model, test_loader, device)
    fedargs.tb.add_scalar("Accuracy/" + client, test_output["accuracy"], epoch)
    log.jsoninfo(test_output, "Test Outut after Epoch {} of {} for Client {}".format(epoch, fedargs.epochs, client))

In [18]:
import time
start_time = time.time()

# Initialize Random Model
global_model = nn.ModelMNIST()

# Federated Training
for _epoch in tqdm(range(fedargs.epochs)):

    epoch = _epoch + 1
    log.info("Federated Training Epoch {} of {}".format(epoch, fedargs.epochs))
    
    # Server, Consume and Average, epoch passed is actually prev epoch, for which we want to consume updates
    rcvd_models = ctp.consume_model(server, fedargs.topic, global_model, epoch - 1)
    log.info("Server received {} model updates from {}".format(len(rcvd_models), list(rcvd_models.keys())))
    if len(rcvd_models) != 0:
        global_model = fl.federated_avg(rcvd_models)
        
    ctp.produce_model(server, fedargs.server_topic, global_model, epoch)
        
    # Gloabal Test
    log.modeldebug(global_model, "Epoch {} of {} : Server Update".format(epoch, fedargs.epochs))
    global_test_output = fl.eval(global_model, test_loader, device)
    fedargs.tb.add_scalar("Gloabl Accuracy/", global_test_output["accuracy"], epoch)
    log.jsoninfo(global_test_output, "Gloabl Test Outut after Epoch {} of {}".format(epoch, fedargs.epochs))
    
    # Clients
    tasks = [process(client, epoch, ctp, global_model,
                     clients_info[client]['train_loader'],
                     clients_info[client]['test_loader'],
                     fedargs, device) for client in clients]
    await asyncio.wait(tasks)

print(time.time() - start_time)

  0%|          | 0/10 [00:00<?, ?it/s]2021-08-02 11:59:31,409 - <ipython-input-18-6cf3b740481c>::<module>(l:11) : Federated Training Epoch 1 of 10 [MainProcess : MainThread (INFO)]
2021-08-02 11:59:41,444 - <ipython-input-18-6cf3b740481c>::<module>(l:15) : Server received 0 model updates from [] [MainProcess : MainThread (INFO)]
2021-08-02 11:59:41,774 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx-cs. ^C to exit. [MainProcess : MainThread (INFO)]
2021-08-02 11:59:41,969 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : MainThread (INFO)]
2021-08-02 11:59:42,841 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(server)' successfully produced to pyflx-cs [0] at offset 57 [MainProcess : MainThread (INFO)]
2021-08-02 11:59:48,835 - <ipython-input-18-6cf3b740481c>::<module>(l:25) : Gloabl 

2021-08-02 12:00:24,866 - <ipython-input-17-2e03a472c65d>::process(l:40) : Test Outut after Epoch 1 of 10 for Client bladecluster.iitp.org(3) {
    "accuracy": 89.55,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1791,
    "test_loss": 0.38545166277885434
} [MainProcess : asyncio_2 (INFO)]
2021-08-02 12:00:24,894 - <ipython-input-17-2e03a472c65d>::process(l:40) : Test Outut after Epoch 1 of 10 for Client bladecluster.iitp.org(4) {
    "accuracy": 90.14999999999999,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1803,
    "test_loss": 0.3726441934108734
} [MainProcess : asyncio_3 (INFO)]
2021-08-02 12:00:25,047 - <ipython-input-17-2e03a472c65d>::process(l:40) : Test Outut after Epoch 1

2021-08-02 12:01:15,624 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(5)' successfully produced to pyflx-c [0] at offset 173 [MainProcess : asyncio_3 (INFO)]
2021-08-02 12:01:16,970 - <ipython-input-17-2e03a472c65d>::process(l:40) : Test Outut after Epoch 2 of 10 for Client bladecluster.iitp.org(1) {
    "accuracy": 92.45,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1849,
    "test_loss": 0.2725658543109894
} [MainProcess : asyncio_0 (INFO)]
2021-08-02 12:01:17,780 - <ipython-input-17-2e03a472c65d>::process(l:40) : Test Outut after Epoch 2 of 10 for Client bladecluster.iitp.org(3) {
    "accuracy": 92.35,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "

2021-08-02 12:02:06,686 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_4 (INFO)]
2021-08-02 12:02:06,726 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(2)' successfully produced to pyflx-c [0] at offset 176 [MainProcess : asyncio_1 (INFO)]
2021-08-02 12:02:07,222 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx-c. ^C to exit. [MainProcess : asyncio_3 (INFO)]
2021-08-02 12:02:07,377 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_3 (INFO)]
2021-08-02 12:02:07,882 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(4)' successfully produced to pyflx-c [0] at offset 177 [MainProcess : asyncio_4 (INFO)]
2021-08-02 12:02:07,890 - /home/ha

2021-08-02 12:02:56,445 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_5 (INFO)]
2021-08-02 12:02:56,964 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx-c. ^C to exit. [MainProcess : asyncio_0 (INFO)]
2021-08-02 12:02:57,134 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_0 (INFO)]
2021-08-02 12:02:57,596 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(2)' successfully produced to pyflx-c [0] at offset 180 [MainProcess : asyncio_5 (INFO)]
2021-08-02 12:02:57,598 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx-c. ^C to exit. [MainProcess : asyncio_3 (INFO)]
2021-08-02 12:02:57,995 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_produce

...

2021-08-02 12:05:41,636 - <ipython-input-17-2e03a472c65d>::process(l:40) : Test Outut after Epoch 7 of 10 for Client bladecluster.iitp.org(1) {
    "accuracy": 97.1,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1942,
    "test_loss": 0.1032664422094822
} [MainProcess : asyncio_3 (INFO)]
2021-08-02 12:05:41,960 - <ipython-input-17-2e03a472c65d>::process(l:40) : Test Outut after Epoch 7 of 10 for Client bladecluster.iitp.org(3) {
    "accuracy": 96.95,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1939,
    "test_loss": 0.1091897641569376
} [MainProcess : asyncio_5 (INFO)]
2021-08-02 12:05:42,243 - <ipython-input-17-2e03a472c65d>::process(l:40) : Test Outut after Epoch 7 of 10 for Cli

2021-08-02 12:06:32,561 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_4 (INFO)]
2021-08-02 12:06:32,995 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx-c. ^C to exit. [MainProcess : asyncio_2 (INFO)]
2021-08-02 12:06:33,183 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_2 (INFO)]
2021-08-02 12:06:33,407 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(4)' successfully produced to pyflx-c [0] at offset 208 [MainProcess : asyncio_4 (INFO)]
2021-08-02 12:06:33,742 - <ipython-input-17-2e03a472c65d>::process(l:40) : Test Outut after Epoch 8 of 10 for Client bladecluster.iitp.org(1) {
    "accuracy": 97.1,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instance

2021-08-02 12:07:25,614 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_4 (INFO)]
2021-08-02 12:07:25,667 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_3 (INFO)]
2021-08-02 12:07:25,914 - <ipython-input-17-2e03a472c65d>::process(l:40) : Test Outut after Epoch 9 of 10 for Client bladecluster.iitp.org(1) {
    "accuracy": 97.45,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1949,
    "test_loss": 0.09076882215589285
} [MainProcess : asyncio_0 (INFO)]
2021-08-02 12:07:26,223 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx-c. ^C to exit. [MainProcess : asyncio_5 (INFO)]
2021-08-02 12:07:26,238 - /home/harsh_1921cs01/hub/F3IA/f

2021-08-02 12:07:56,636 - <ipython-input-17-2e03a472c65d>::process(l:16) : Client bladecluster.iitp.org(1) received 1 model updates from ['bladecluster.iitp.org(server)'] [MainProcess : asyncio_0 (INFO)]
2021-08-02 12:07:56,636 - <ipython-input-17-2e03a472c65d>::process(l:16) : Client bladecluster.iitp.org(4) received 1 model updates from ['bladecluster.iitp.org(server)'] [MainProcess : asyncio_5 (INFO)]
2021-08-02 12:07:56,637 - <ipython-input-17-2e03a472c65d>::process(l:16) : Client bladecluster.iitp.org(3) received 1 model updates from ['bladecluster.iitp.org(server)'] [MainProcess : asyncio_3 (INFO)]
2021-08-02 12:08:13,668 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx-c. ^C to exit. [MainProcess : asyncio_0 (INFO)]
2021-08-02 12:08:13,804 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_0 (INFO)]
2021-08-02 12:08:14,358 - /home/harsh_1921cs0

528.9992275238037
